In [ ]:
import arcpy
from arcpy.sa import *
import os
import time
arcpy.env.overwriteOutput = True

In [ ]:
# load input features and make layer
version='v10_240710'
output_dir = r'J:\lakemapping\postprocess\v10_240710'
after_mask_gdb=os.path.join(output_dir,'3_polygon_after_rivermask.gdb')
stastics_gdb=os.path.join(output_dir,'0_summary_stastics.gdb')
mwBL_gdb=os.path.join(output_dir,'4_polygon_afm_mergeWith_BigLake.gdb')
mwBG_gdb=os.path.join(output_dir,'5_polygon_afm_mwBL_BigGLAKES.gdb')
excel_dir=os.path.join(output_dir,'stastics_excel')
if not os.path.exists(mwBL_gdb):
    arcpy.CreateFileGDB_management(output_dir,'4_polygon_afm_mergeWith_BigLake.gdb')
if not os.path.exists(mwBG_gdb):
    arcpy.CreateFileGDB_management(output_dir,'5_polygon_afm_mwBL_BigGLAKES.gdb')
if not os.path.exists(stastics_gdb):
    arcpy.CreateFileGDB_management(output_dir,'0_summary_stastics.gdb')

In [ ]:
a_iwBL_merge_with_BL_raw=os.path.join(after_mask_gdb,'total_lakes_afm_iwBL_merge_with_BigLakes_raw')
a_iwBL_merge_with_BL=os.path.join(after_mask_gdb,'total_lakes_afm_iwBL_merge_with_BigLakes')
a_iwBL_merge_with_BL_raw_lyr='lakes_afm_iwBL_mw_BL_raw'
a_iwBL_merge_with_BL_lyr='lakes_afm_iwBL_mw_BL'
b_iwBG_merge_with_BG_raw=os.path.join(mwBL_gdb,'lakes_afm_mwBL_iwBG_merge_with_BG_raw')
b_iwBG_merge_with_BG=os.path.join(mwBL_gdb,'lakes_afm_mwBL_iwBG_merge_with_BG')
b_iwBG_merge_with_BG_SJ=os.path.join(mwBL_gdb,'b_iwBG_merge_with_BG_SJ')
b_iwBG_merge_with_BG_raw_lyr='lakes_afm_mwBL_iwBG_mw_BG_raw'
b_iwBG_merge_with_BG_lyr='lakes_afm_mwBL_iwBG_mw_BG'
b_iwBG_merge_with_BG_SJ_lyr='lakes_afm_mwBL_iwBG_mw_BG_SJ'

In [ ]:
seven_continents=['Asia','Africa','Europe','Oceania_Anta','North_America','South_America']

In [ ]:
def selectByLocation(inputFeature,overlapType,selectionFeature,selectionType='NEW_SELECTION'):
    arcpy.SelectLayerByLocation_management(inputFeature,overlapType,selectionFeature,selection_type=selectionType)
    
def selectByAttribute(inputFeature,selectionType='NEW_SELECTION',code=''):
    arcpy.management.SelectLayerByAttribute(inputFeature,selectionType,code)
    
def calculateField(inputFeature,field,code):
    arcpy.CalculateField_management(inputFeature, field,code)

In [ ]:
region_list=['eu','af','oc','island','greenland','sa_1','sa_2']
for i in range(1,10):
    region_list.append(f'as_{i}')
    
for i in range(1,7):
    region_list.append(f'na_{i}')

In [ ]:
big_lakes=os.path.join(after_mask_gdb,f'total_big_lakes')
big_lakes_lyr=f'big_lakes'
GLAKES_gte1=os.path.join(mwBL_gdb,'GLAKES_gte1')
GLAKES_gte1_singlepart=os.path.join(mwBL_gdb,'GLAKES_gte1_singlepart')
GLAKES_gte1_point=os.path.join(mwBL_gdb,'GLAKES_gte1_point')
GLAKES_lt1=os.path.join(mwBL_gdb,'GLAKES_lt1')
GLAKES_gte1_lyr='GLAKES_gte1'
GLAKES_lt1_lyr='GLAKES_lt1'
GLAKES_gte1_singlepart_lyr='GLAKES_gte1_singlepart'
GLAKES_gte1_point_lyr='GLAKES_gte1_point'

In [ ]:
continent=r'D:\lakemapping\0_auxiliary_data\World_Continents\World_Continents.shp'
continent_lyr='continent'
arcpy.MakeFeatureLayer_management(continent,continent_lyr)

## 1， 对GLAKES进行分类，合并big_lakes

In [ ]:
GLAKES=r'D:\lakemapping\0_auxiliary_data\GLAKES\GLAKES.gdb\GLAKES'
arcpy.MakeFeatureLayer_management(GLAKES,'GLAKES')
selectByAttribute('GLAKES','NEW_SELECTION','"Area_bound" >= 1')
arcpy.conversion.ExportFeatures('GLAKES',GLAKES_gte1)
selectByAttribute('GLAKES','SWITCH_SELECTION','')
arcpy.conversion.ExportFeatures('GLAKES',GLAKES_lt1)

In [ ]:
arcpy.MakeFeatureLayer_management(GLAKES_gte1,GLAKES_gte1_lyr)
arcpy.MakeFeatureLayer_management(GLAKES_lt1,GLAKES_lt1_lyr)

In [ ]:
# arcpy.management.MultipartToSinglepart(GLAKES_gte1_lyr, GLAKES_gte1_singlepart)
# arcpy.MakeFeatureLayer_management(GLAKES_gte1_singlepart,GLAKES_gte1_singlepart_lyr)
arcpy.management.FeatureToPoint(GLAKES_gte1_singlepart,GLAKES_gte1_point,'INSIDE')
arcpy.MakeFeatureLayer_management(GLAKES_gte1_point,GLAKES_gte1_point_lyr)

In [ ]:
big_lake_list=[]
for region in region_list:
    polygon_big_lake=os.path.join(after_mask_gdb,'big_lakes_'+region)
    arcpy.MakeFeatureLayer_management(polygon_big_lake,'big_lakes_'+region)
    big_lake_list.append('big_lakes_'+region)
arcpy.Merge_management(big_lake_list,big_lakes)

## 2，将mask后结果与big_lakes合并，得到的数据存在4_mwBL_gdb

In [ ]:
arcpy.MakeFeatureLayer_management(big_lakes,big_lakes_lyr)

In [ ]:
## 添加 Continent 及 iwBL 属性
start=time.time()
input_table_afm_list=[]
for region in region_list:
    lakes_afm=os.path.join(after_mask_gdb,'lakes_'+region+'_afterMask')
    lakes_afm_lyr='lakes_'+region+'_afm'
    arcpy.MakeFeatureLayer_management(lakes_afm,lakes_afm_lyr)
#     arcpy.AddField_management(lakes_afm_lyr, 'Continent', "Text")
#     arcpy.AddField_management(lakes_afm_lyr, 'iwBL', "Short")
    selectByLocation(lakes_afm_lyr,'INTERSECT',big_lakes_lyr)
    calculateField(lakes_afm_lyr,'iwBL',1)
#     for i in range(0,6):
#         continent=seven_continents[i]
#         selectByAttribute(continent_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
#         selectByAttribute(lakes_afm_lyr,'NEW_SELECTION',"Continent IS NULL")
#         selectByLocation(lakes_afm_lyr,'HAVE_THEIR_CENTER_IN',continent_lyr,'SUBSET_SELECTION')
#         calculateField(lakes_afm_lyr,'Continent',f"'{continent}'")
    selectByAttribute(lakes_afm_lyr,'NEW_SELECTION','')
#     out_table=os.path.join(stastics_gdb,f'{region}_afm_by_Continent_stastics')
    out_table=os.path.join(stastics_gdb,f'{region}_afm_by_Continent_stastics_{version}')
    input_table_afm_list.append(out_table)
    arcpy.analysis.Statistics(lakes_afm_lyr, out_table, [['area',"sum"]], ['Continent','iwBL']) 
#     selectByAttribute(lakes_afm_lyr,'NEW_SELECTION','iwBL IS NULL')
#     calculateField(lakes_afm_lyr,'iwBL',0)
print(input_table_afm_list)
end=time.time()
print(f'time:{end-start}s')

In [ ]:
region='sa_2'
lakes_afm=os.path.join(after_mask_gdb,'lakes_'+region+'_afterMask')
lakes_afm_lyr='lakes_'+region+'_afm'
arcpy.MakeFeatureLayer_management(lakes_afm,lakes_afm_lyr)
# arcpy.AddField_management(lakes_afm_lyr, 'iwBL', "Short")
selectByLocation(lakes_afm_lyr,'INTERSECT',big_lakes_lyr)
# calculateField(lakes_afm_lyr,'iwBL',1)
# selectByAttribute(lakes_afm_lyr,'NEW_SELECTION','iwBL IS NULL')
# calculateField(lakes_afm_lyr,'iwBL,0)
# selectByAttribute(lakes_afm_lyr,'NEW_SELECTION','')
# out_table=os.path.join(stastics_gdb,f'{region}_afm_by_Continent_stastics_{version}')
# # input_table_afm_list.append(out_table)
# arcpy.analysis.Statistics(lakes_afm_lyr, out_table, [['area',"sum"]], ['Continent','iwBL']) 

In [ ]:
# Output_Excel_File=r'J:\lakemapping\postprocess\v10_240710\sum_afm_classify_by_Continents_stastics.xls'
file_name=f'sum_lakes_afm_classify_by_Continents_stastics_{version}.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(input_table_afm_list, Output_Excel_File)

In [ ]:
j=16

In [ ]:
# region='island'
# lakes_afm=os.path.join(after_mask_gdb,'lakes_'+region+'_afterMask')
# lakes_afm_lyr='lakes_'+region+'_afm'
# arcpy.MakeFeatureLayer_management(lakes_afm,lakes_afm_lyr)
#     arcpy.AddField_management(lakes_afm_lyr, 'Continent', "Text")
#     arcpy.AddField_management(lakes_afm_lyr, 'iwBL', "Short")
#     selectByLocation(lakes_afm_lyr,'INTERSECT',big_lakes_lyr)
#     calculateField(lakes_afm_lyr,'iwBL',1)
# for i in range(0,6):
#     continent=seven_continents[i]
#     selectByAttribute(continent_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
#     selectByAttribute(lakes_afm_lyr,'NEW_SELECTION',"Continent IS NULL")
#     selectByLocation(lakes_afm_lyr,'HAVE_THEIR_CENTER_IN',continent_lyr,'SUBSET_SELECTION')
#     calculateField(lakes_afm_lyr,'Continent',f"'{continent}'")
# selectByAttribute(lakes_afm_lyr,'NEW_SELECTION','')
# out_table=os.path.join(stastics_gdb,f'{region}_afm_by_Continent_stastics')
# input_table_afm_list.append(out_table)
# arcpy.analysis.Statistics(lakes_afm_lyr, out_table, [['area',"sum"]], ['Continent']) 
# j=j+1
# arcpy.Delete_management(lakes_afm_lyr)

In [ ]:
afm_list=[]
for region in region_list:
    lakes_afm_lyr='lakes_'+region+'_afm'
    lakes_afm=os.path.join(after_mask_gdb,'lakes_'+region+'_afterMask')
    arcpy.MakeFeatureLayer_management(lakes_afm,lakes_afm_lyr)
    afm_list.append(lakes_afm_lyr)
    selectByAttribute(lakes_afm_lyr,'NEW_SELECTION',"iwBL = 1")

In [ ]:
#把iwBL与big lake文件合并，并融合
merge_list=afm_list.copy()
merge_list.append(big_lakes_lyr)
print(merge_list)
start=time.time()
arcpy.Merge_management(merge_list,a_iwBL_merge_with_BL_raw)
arcpy.MakeFeatureLayer_management(a_iwBL_merge_with_BL_raw, a_iwBL_merge_with_BL_raw_lyr)#global_lakes_afm_merged_with_GBL
end=time.time()
print(f'time:{end-start}s')

In [ ]:
# start=time.time()
# print('start Dissolve')
# arcpy.Dissolve_management(a_iwBL_merge_with_BL_raw_lyr, a_iwBL_merge_with_BL,"", "", "SINGLE_PART")#dissolve copylake
# arcpy.MakeFeatureLayer_management(a_iwBL_merge_with_BL, a_iwBL_merge_with_BL_lyr)
# arcpy.AddField_management(a_iwBL_merge_with_BL_lyr,"area","Double")
arcpy.CalculateField_management(a_iwBL_merge_with_BL_lyr, "area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
end=time.time()
print(f'time:{end-start}s')

In [ ]:
arcpy.MakeFeatureLayer_management(a_iwBL_merge_with_BL, a_iwBL_merge_with_BL_lyr)

In [ ]:
arcpy.AddField_management(a_iwBL_merge_with_BL_lyr, 'Continent', "Text")
for i in range(0,6):
    continent=seven_continents[i]
    selectByAttribute(continent_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
    selectByAttribute(a_iwBL_merge_with_BL_lyr,'NEW_SELECTION',"Continent IS NULL")
    selectByLocation(a_iwBL_merge_with_BL_lyr,'HAVE_THEIR_CENTER_IN',continent_lyr,'SUBSET_SELECTION')
    calculateField(a_iwBL_merge_with_BL_lyr,'Continent',f"'{continent}'")

In [ ]:
# continent_region_list=[[1,2,4,8,9,10,11,12,13,14,15,16],[2,4,16],[1,4,5,16],[3,4,15],[4,5,6,12,17,18,19,20,21,22],[4,6,7]]
# b_lyr_list=[]
# input_table_b_list=[]
for i in range(0,6):
    continent=seven_continents[i]
#     continent_merge_list=[a_iwBL_merge_with_BL_lyr]
#     for j in continent_region_list[i]:
#         region_id=j-1#算错了
#         selectByAttribute(afm_list[region_id],'NEW_SELECTION',f"Continent = '{continent}' AND iwBL = 0")
#         continent_merge_list.append(afm_list[region_id])
#     print(continent_merge_list)
    
#     selectByAttribute(a_iwBL_merge_with_BL_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
#     b_lake_afm_merge_with_BL=os.path.join(mwBL_gdb,f'b{i+1}_{continent}_polygon_afm_mwBL')
    b_lake_afm_merge_with_BL_lyr=f'b{i+1}_{continent}_lake_afm_merge_with_BL'
#     b_lyr_list.append(b_lake_afm_merge_with_BL_lyr)
#     arcpy.Merge_management(continent_merge_list,b_lake_afm_merge_with_BL)
#     arcpy.MakeFeatureLayer_management(b_lake_afm_merge_with_BL, b_lake_afm_merge_with_BL_lyr)
#     arcpy.AddField_management(b_lake_afm_merge_with_BL_lyr, 'iwBG', "Short")
#     selectByLocation(b_lake_afm_merge_with_BL_lyr,'INTERSECT',GLAKES_lt1_lyr)
#     calculateField(b_lake_afm_merge_with_BL_lyr,'iwBG',1)
#     selectByLocation(b_lake_afm_merge_with_BL_lyr,'INTERSECT',GLAKES_gte1_lyr,'SUBSET_SELECTION')
#     calculateField(b_lake_afm_merge_with_BL_lyr,'iwBG',3)
#     selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','iwBG IS NULL')
#     selectByLocation(b_lake_afm_merge_with_BL_lyr,'INTERSECT',GLAKES_gte1_lyr,'SUBSET_SELECTION')
#     calculateField(b_lake_afm_merge_with_BL_lyr,'iwBG',2)
#     selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','iwBG IS NULL')
#     calculateField(b_lake_afm_merge_with_BL_lyr,'iwBG',0)
#     out_table=os.path.join(stastics_gdb,f'{continent}_lakes_merge_with_BL_stastics_{version}')
#     input_table_b_list.append(out_table)
#     selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','')
#     arcpy.analysis.Statistics(b_lake_afm_merge_with_BL_lyr, out_table, [['area',"sum"]], ['iwBG']) 

    if (i==0 or i==2):
        gdb=r'J:\lakemapping\postprocess\dataset_for_check\Asia_Europe\dataset_Asia_Europe.gdb'
    elif (i==1 or i==3 or i==5):
        gdb=r'J:\lakemapping\postprocess\dataset_for_check\Africa_Oceania_South_America\Africa_Oceania_South_America.gdb'
    else:
        gdb=r'J:\lakemapping\postprocess\dataset_for_check\North_America\North_America.gdb'
    new_b_lake_afm_merge_with_BL=os.path.join(gdb,f'b{i+1}_{continent}_polygon_afm_mwBL')
    field_list=['area','operate','iwBG']
    fieldmappings = arcpy.FieldMappings()
    fieldmappings_all = arcpy.FieldMappings()
    fieldmappings_all.addTable(b_lake_afm_merge_with_BL_lyr)
    for field_i in field_list:
        field_idx = fieldmappings_all.findFieldMapIndex(field_i)
        fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
    arcpy.conversion.ExportFeatures(b_lake_afm_merge_with_BL_lyr,new_b_lake_afm_merge_with_BL,field_mapping=fieldmappings)

In [ ]:
selectByLocation(b_lake_afm_merge_with_BL_lyr,'INTERSECT',GLAKES_lt1_lyr)

In [ ]:
# out_table=os.path.join(stastics_gdb,f'{continent}_lakes_merge_with_BL_stastics_{version}')
# # input_table_b_list.append(out_table)
# selectByAttribute(b_lake_afm_merge_with_BL_lyr,'NEW_SELECTION','')
# arcpy.analysis.Statistics(b_lake_afm_merge_with_BL_lyr, out_table, [['area',"sum"]], ['iwBG']) 

In [ ]:
file_name=f'sum_lakes_afm_mwBL_stastics_{version}.xls'
Output_Excel_File=os.path.join(excel_dir,file_name)
arcpy.conversion.TableToExcel(input_table_b_list, Output_Excel_File)

## 3，将b结果与big_GLAKES合并，得到的数据存在mwBGgdb

In [ ]:
#3.1 把iwBG与big GLAKES文件合并，并融合
merge_list_BG=b_lyr_list.copy()
merge_list_BG.append(GLAKES_gte1_lyr)
print(merge_list_BG)

start=time.time()
for i in range(0,6):
    b_lyr=b_lyr_list[i]
    selectByAttribute(b_lyr,'NEW_SELECTION',"iwBG > 1")
    
arcpy.Merge_management(merge_list_BG,b_iwBG_merge_with_BG_raw)
arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG_raw, b_iwBG_merge_with_BG_raw_lyr)#global_lakes_afm_merged_with_GBL
end=time.time()
print(f'time:{end-start}s')

In [ ]:
start=time.time()
print('start Dissolve')
arcpy.Dissolve_management(b_iwBG_merge_with_BG_raw_lyr, b_iwBG_merge_with_BG,"", "", "SINGLE_PART")# dissolve copylake
arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG, b_iwBG_merge_with_BG_lyr)
arcpy.AddField_management(b_iwBG_merge_with_BG_lyr,"area","Double")
arcpy.CalculateField_management(b_iwBG_merge_with_BG_lyr, "area",  "!shape.geodesicArea@SQUAREKILOMETERS!", "PYTHON_9.3")
end=time.time()
print(f'time:{end-start}s')

In [ ]:
arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG, b_iwBG_merge_with_BG_lyr)

In [ ]:
field_list=['area','Lake_id']
fieldmappings = arcpy.FieldMappings()
fieldmappings_all = arcpy.FieldMappings()
fieldmappings_all.addTable(b_iwBG_merge_with_BG_lyr)#未mask
fieldmappings_all.addTable(GLAKES_gte1_lyr)
for field_i in field_list:
        field_idx = fieldmappings_all.findFieldMapIndex(field_i)
        fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))

print('Conduct spatial join')
arcpy.SpatialJoin_analysis(b_iwBG_merge_with_BG_lyr, GLAKES_gte1_point_lyr, b_iwBG_merge_with_BG_SJ, "JOIN_ONE_TO_ONE",
                           "KEEP_ALL", fieldmappings, "CONTAINS")


In [ ]:
# b_iwBG_merge_with_BG_SJ_lyr='b_iwBG_merge_with_BG_SJ_lyr'
# arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG_SJ,b_iwBG_merge_with_BG_SJ_lyr)

In [ ]:
b_iwBG_merge_with_BG_SJ_SJ=os.path.join(mwBL_gdb,'b_iwBG_merge_with_BG_SJ_SJ')
b_iwBG_merge_with_BG_SJ_SJ_lyr='b_iwBG_merge_with_BG_SJ_SJ'
selectByAttribute(b_iwBG_merge_with_BG_SJ_lyr,,'NEW_SELECTION','Join_Count = 0')
print('Conduct spatial join')
arcpy.SpatialJoin_analysis(b_iwBG_merge_with_BG_SJ_lyr, GLAKES_gte1_lyr, b_iwBG_merge_with_BG_SJ_SJ, "JOIN_ONE_TO_MORE",
                           "KEEP_ALL", fieldmappings, "LARGEST_OVERLAP")
arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG_SJ_SJ,b_iwBG_merge_with_BG_SJ_SJ_lyr)

In [ ]:
# arcpy.management.AddJoin(b_iwBG_merge_with_BG_SJ_lyr,'OBJECTID',b_iwBG_merge_with_BG_SJ_SJ_lyr,'TARGET_FID',"KEEP_COMMON")

In [ ]:
arcpy.AddField_management(b_iwBG_merge_with_BG_lyr, 'Continent', "Text")
for i in range(1,6):
    continent=seven_continents[i]
    selectByAttribute(continent_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
    selectByAttribute(b_iwBG_merge_with_BG_lyr,'NEW_SELECTION',"Continent IS NULL")
    selectByLocation(b_iwBG_merge_with_BG_lyr,'HAVE_THEIR_CENTER_IN',continent_lyr,'SUBSET_SELECTION')
    calculateField(b_iwBG_merge_with_BG_lyr,'Continent',f"'{continent}'")

In [ ]:
b_iwBG_merge_with_BG_SJ_dissolve=os.path.join(mwBL_gdb,'b_iwBG_merge_with_BG_SJ_dissolve')
b_iwBG_merge_with_BG_SJ_dissolve_lyr='b_iwBG_merge_with_BG_SJ_dissolve'
arcpy.Dissolve_management(b_iwBG_merge_with_BG_lyr, b_iwBG_merge_with_BG_dissolve,"Lake_id", "", "SINGLE_PART")#dissolve copylake
arcpy.MakeFeatureLayer_management(b_iwBG_merge_with_BG_SJ_dissolve, b_iwBG_merge_with_BG_SJ_dissolve_lyr)

In [ ]:
arcpy.AddField_management(b_iwBG_merge_with_BG_SJ_dissolve_lyr, 'Continent', "Text")
for i in range(0,1):
    continent=seven_continents[i]
    selectByAttribute(continent_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
    selectByAttribute(b_iwBG_merge_with_BG_SJ_dissolve_lyr,'NEW_SELECTION',"Continent IS NULL")
    selectByLocation(b_iwBG_merge_with_BG_SJ_dissolve_lyr,'HAVE_THEIR_CENTER_IN',continent_lyr,'SUBSET_SELECTION')
    calculateField(b_iwBG_merge_with_BG_SJ_dissolve_lyr,'Continent',f"'{continent}'")

In [ ]:
## 3.2 将niwGB与 3.1的结果合并生成c
for i in range(0,6):
    b_lyr=b_lyr_list[i]
    continent=seven_continents[i]
    selectByAttribute(b_lyr,'NEW_SELECTION',"iwBG = 0")
    selectByAttribute(b_iwBG_merge_with_BG_SJ_dissolve_lyr,'NEW_SELECTION',f"Continent = '{continent}'")
    c_lake_afm_mw_BL_BG=os.path.join(mwBG_gdb,f'c{i+1}_{continent}_polygon_afm_mwBL_BG')
    c_lake_afm_mw_BL_BG_lyr=f'c{i+1}_{continent}_lake_afm_mw_BL_BG'
    arcpy.Merge_management([b_lyr,b_iwBG_merge_with_BG_SJ_dissolve_lyr],c_lake_afm_mw_BL_BG,fieldmappings)
    arcpy.MakeFeatureLayer_management(c_lake_afm_mw_BL_BG,c_lake_afm_mw_BL_BG_lyr)
    #export to other place
#     if (i==0 or i==2):
#         gdb=r'J:\lakemapping\postprocess\dataset_for_check\Asia_Europe\dataset_Asia_Europe.gdb'
#     elif (i==1 or i==3 or i==5):
#         gdb=r'J:\lakemapping\postprocess\dataset_for_check\Africa_Oceania_South_America\Africa_Oceania_South_America.gdb'
#     else:
#         gdb=r'J:\lakemapping\postprocess\dataset_for_check\North_America\North_America.gdb'
#     new_c_lake_afm_mw_BL_BG=os.path.join(gdb,f'c{i+1}_{continent}_polygon_afm_mwBL_BG')
#     field_list=['area','iwBG','operate','Lake_id']
#     fieldmappings = arcpy.FieldMappings()
#     fieldmappings_all = arcpy.FieldMappings()
#     fieldmappings_all.addTable(b_lyr)#未mask
#     fieldmappings_all.addTable(b_iwBG_merge_with_BG_SJ_dissolve_lyr)
#     for field_i in field_list:
#         field_idx = fieldmappings_all.findFieldMapIndex(field_i)
#         fieldmappings.addFieldMap(fieldmappings_all.getFieldMap(field_idx))
#     arcpy.conversion.ExportFeatures(c_lake_afm_mw_BL_BG_lyr,new_c_lake_afm_mw_BL_BG,field_mapping=fieldmappings)
    

In [ ]:
# input_table_mwBL_list=[]
# input_table_mwBL_BG_list=[]
# for i in range(0,6):
#     b_lyr=b_lyr_list[i]
#     continent=seven_continents[i]
#     out_table_1=os.path.join(stastics_gdb,f'b{i+1}_{continent}_afm_mwBL_by_Continent_stastics')
#     input_table_mwBL_list.append(out_table_1)
#     c_lake_afm_mw_BL_BG=os.path.join( Cgdb,f'c{i+1}_{continent}_polygon_afm_mwBL_BG')
#     c_lake_afm_mw_BL_BG_lyr=f'c{i+1}_{continent}_lake_afm_mw_BL_BG'
# #     arcpy.MakeFeatureLayer_management(c_lake_afm_mw_BL_BG,c_lake_afm_mw_BL_BG_lyr)
#     out_table=os.path.join(stastics_gdb,f'c{i+1}_{continent}_afm_mwBL_BG_by_Continent_stastics')
#     input_table_mwBL_BG_list.append(out_table)
# #     arcpy.analysis.Statistics(c_lake_afm_mw_BL_BG_lyr, out_table, [['area',"sum"]]) 
#     out_table_2=os.path.join(stastics_gdb,f'c{i+1}_{continent}_afm_mwBL_BG_by_Continent_stastics')
#     input_table_mwBL_BG_list.append(out_table_2)
    
# Output_Excel_File=r'J:\lakemapping\postprocess\v10_240710\sum_afm_mwBL_classify_by_Continents_stastics.xls'
# Output_Excel_File_2=r'J:\lakemapping\postprocess\v10_240710\sum_afm_mwBL_BG_classify_by_Continents_stastics.xls'
# print(input_table_mwBL_BG_list)
# arcpy.conversion.TableToExcel(input_table_mwBL_list, Output_Excel_File)
# arcpy.conversion.TableToExcel(input_table_mwBL_BG_list, Output_Excel_File_2)